In [2]:
# %matplotlib inline
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from esmpy.models import edxs
from esmpy.models.EDXS_function import print_concentrations_from_W
from esmpy.conf import DATASETS_PATH
from pathlib import Path
from esmpy.datasets import spim
import esmpy.tables_utils as tu
from esmpy.estimators import SmoothNMF

In [3]:
spim = hs.load("eds_rebin.hspy")

spim.set_signal_type("EDS_ESMPY")
spim.set_additional_parameters(thickness = 80e-7,density = 2.1,detector_type = "Det_eff.txt",width_slope = 0.01,width_intercept = 0.065,xray_db = "300keV_corentin_xrays.json")


# spim.set_analysis_parameters(beam_energy = 300,
# azimuth_angle = 0.0,
# elevation_angle = 35.0,
# elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti', 'Zr'],
# tilt_stage = -6.071,
# thickness = 80e-7,
# density = 2.1,
# detector_type = "Det_eff.txt",
# width_slope = 0.01,
# width_intercept = 0.065,
# xray_db = "300keV_corentin_xrays.json")

cropped_spim = spim.inav[:,:]

G = cropped_spim.build_G("bremsstrahlung")
est = SmoothNMF( n_components = 7,tol=0.000001, max_iter = 100000, G = G,hspy_comp = True)

cropped_spim.decomposition(algorithm=est)

/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/hyperspy/misc/utils.py:471: VisibleDeprecationWarning: Use of the `binned` attribute in metadata is going to be deprecated in v2.0. Set the `axis.is_binned` attribute instead. 
  warnings.warn(
/home/adrien/anaconda3/envs/esmpy/lib/python3.10/site-packages/hyperspy/io.py:560: VisibleDeprecationWarning: Loading old file version. The binned attribute has been moved from metadata.Signal to axis.is_binned. Setting this attribute for all signal axes instead.
  warnings.warn('Loading old file version. The binned attribute '


Dicotomy stopped for maximum number of iterations
It 10 / 100000: loss 2.588,  0.806 it/s
Dicotomy stopped for maximum number of iterations
Dicotomy stopped for maximum number of iterations
Dicotomy stopped for maximum number of iterations
Dicotomy stopped for maximum number of iterations
Dicotomy stopped for maximum number of iterations


/home/adrien/travail/esmpy/esmpy/updates.py:46: RuntimeWarning: divide by zero encountered in true_divide
  return np.sum(num / (x + denum), axis=0) - 1


Dicotomy stopped for maximum number of iterations
Stopped after 15 iterations in 0.0 minutes and 19.0 seconds.
Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SmoothNMF()
  output_dimension=None
  centre=None
scikit-learn estimator:
SmoothNMF()


In [6]:
cropped_spim.plot_decomposition_results()

fW = est.W_
print_concentrations_from_W(fW[:-2,:],elements = cropped_spim.metadata.Sample.elements)

In [18]:
input_filename = str("phyllo_spectre_moyen.hspy")
spectrum = hs.load(input_filename)

spectrum.axes_manager[0].offset += +0.000

spectrum.set_signal_type("EDS_ESMPY")
spectrum.set_analysis_parameters(beam_energy = 300,
azimuth_angle = 0.0,
elevation_angle = 18.0,
elements = ['Al', 'C', 'Ca', 'Cl', 'Cr', 'Cu', 'F', 'Fe', 'Ga', 'K', 'Mg', 'Mn', 'N', 'Na', 'Ni', 'O', 'P', 'Pt', 'S', 'Si', 'Ti', 'Zr'],
tilt_stage = 0.001,
thickness = 80e-7,
density = 2.1,

detector_type = "Det_eff.txt",
width_slope = 0.01,
width_intercept = 0.065,
xray_db = "300keV_corentin_xrays.json")
# fW = spim.set_fixed_W({"p0" : {"Si" : 0.0},"p1" : {"Fe" : 0.0}, "p2" : {"Mg" : 0.0}})

G = spectrum.build_G("bremsstrahlung",norm = True)
est = SmoothNMF( n_components = 1,tol=0.000001, max_iter = 1000, G = G)

est.fit_transform(spectrum.data[:,np.newaxis])
fW = est.W_
fG = est.G_

It 10 / 1000: loss 0.044,  28.576 it/s
It 20 / 1000: loss 0.044,  27.725 it/s
It 30 / 1000: loss 0.044,  27.658 it/s
It 40 / 1000: loss 0.044,  27.686 it/s
It 50 / 1000: loss 0.044,  27.765 it/s
It 60 / 1000: loss 0.044,  27.746 it/s
It 70 / 1000: loss 0.044,  27.685 it/s
It 80 / 1000: loss 0.044,  27.654 it/s
It 90 / 1000: loss 0.044,  27.635 it/s
exits because of relative change < tol: 1.5390655927805505e-06
Stopped after 99 iterations in 0.0 minutes and 4.0 seconds.


In [21]:
plt.plot(spectrum.data,label="exp")
plt.plot(fG@fW,label="fit")
plt.legend()

In [20]:
print_concentrations_from_W(fW[:-2,:],elements = spectrum.metadata.Sample.elements)

Concentrations report
     p0
Al : 0.0177 
C  : 0.0434 
Ca : 0.0000 
Cl : 0.0007 
Cr : 0.0000 
Cu : 0.0189 
F  : 0.0363 
Fe : 0.0866 
Ga : 0.0044 
K  : 0.0005 
Mg : 0.1844 
Mn : 0.0035 
N  : 0.0070 
Na : 0.0087 
Ni : 0.0019 
O  : 0.3663 
P  : 0.0000 
Pt : 0.0013 
S  : 0.0080 
Si : 0.2096 
Ti : 0.0000 
Zr : 0.0008 


In [31]:
k_names = ["Ag_La","Al_Ka", "Ar_Ka","B_Ka","Bi_Ma","C_Ka","Ca_Ka","Ca_La","Cl_Ka","Cr_Ka",
        "Cr_La","Cu_Ka","Cu_La","F_Ka","Fe_Ka","Fe_La","Fe_Lb3","Fe_Ll","Fe_Ln","Ga_Ka",
        "Ga_La","Ho_La","La_La","K_Ka","Mg_Ka","Mn_Ka","Mn_La","Mg_La","N_Ka","Na_Ka",
        "Ni_Ka","Ni_La","O_Ka","P_Ka","Pb_La","Pt_La","Pt_Ma","S_Ka","Si_Ka","Sn_La",
        "Ti_Ka","Tl_Ma","Zr_Ka","Zr_La","Zn_Ka"]
kfactors = [1,
 1.05,
 1.188,
 5,
 1,
 1.42,
 1.16,
 8.8,
 1.093,
 1.314,
 5.23,
 1.773,
 1.523,
 1,
 1.5,
 2.55,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1.119,
 0.94,
 1.5,
 3.36,
 3.36,
 1.18,
 0.98,
 1.587,
 1.582,
 0.95,
 1.096,
 2.85,
 1,
 1,
 1.18,
 1,
 1,
 1.25,
 1,
 3.65,
 1.49,
 1.96]


In [42]:
table, mdata = tu.load_table("300keV_xrays.json")
n_table, n_mdata = tu.import_k_factors(table,mdata, k_names, kfactors,  ref_name = "Si_Ka")
tu.save_table("300keV_corentin_xrays.json",n_table,n_mdata)